In [1]:
# IMPORTS
##########################

%load_ext autoreload
%autoreload 2

import agent
import environment
import doubledqn
import tools
import memory
import simulation
import multiprocessing
import pandas as pd
import os
import json

import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

import numpy as np
import matplotlib.pyplot as plt
import time
import itertools
from keras import optimizers 

def iter_params(**kwargs):
    keys = kwargs.keys()
    vals = kwargs.values()
    for instance in itertools.product(*vals):
        yield dict(zip(keys, instance))

Using TensorFlow backend.


In [ ]:
# ONE RUN
#############################################

param = { 
    
    "batch_size" : 30, # 30,50,60,[70]
    "target_update_freq" : 5000,#10000], # 10000,20000,30000,[40000]
    "gamma" : 0.99,# # 0.98,0.99,0.995,[0.999]
    "train_freq" : 1, # 2,3,4,[5]
    "max_size" : 100000,#,70000], # 20000,50000,70000,[100000]
    "max_ep_length" : 2000, # 1000,2000,3000,[4000]
    "policy" : "linDecEpsGreedy",#, "epsGreedy"],
    "eps" : 0.2,#, 0.3, 0.1],
    "delta_time" : 10,
    #"optimizer": [optimizers.RMSprop(lr= 0.001), optimizers.Adagrad(), optimizers.Adam()]
}

sumo_RL = simulation.simulator(
                    connection_label = "single_worker",
                    q_network_type = 'simple',
                    target_q_network_type = 'simple',
                    gamma = param["gamma"],
                    target_update_freq = param["target_update_freq"],
                    train_freq = param["train_freq"],
                    num_burn_in = 200,
                    batch_size = param["batch_size"],
                    optimizer = 'adam',
                    loss_func = "mse",
                    max_ep_length = param["max_ep_length"],
                    experiment_id = "check eval",
                    model_checkpoint = True,
                    opt_metric = None,

                   # environment parameters
                    network = "simple", # complex
                    net_file = "simple_cross.net.xml", # "complex_cross.net.xml",
                    route_file = "cross.rou.xml",
                    demand = "nominal",
                    state_shape = (1,15), #(1,11) or (1,29)
                    num_actions = 2, # 2 or 4
                    use_gui = False,
                    delta_time = param["delta_time"],

                   # memory parameters
                    max_size = param["max_size"],

                   # additional parameters

                    policy = param["policy"],
                    eps = param["eps"],
                    num_episodes = 1,
                    monitoring = True,
                    episode_recording = False,
                    hparams = param.keys())

sumo_RL.train()
# agent.load("./logs/First gridsearch/run_144/model_checkpoints/runFirst gridsearch_iter165000.h5")

# agent.ddqn.train(env = agent.env, num_episodes = 100, policy = agent.policy, connection_label = agent.connection_label)

#evaluation_results = agent.evaluate(runs=2, use_gui= False)
#evaluation_results
# data= agent.ddqn.evaluate(env = agent.env, policy = "greedy")

In [ ]:
sumo_RL.evaluate(runs=2)

In [2]:
# GRIDSEARCH
###############################################################

experiment_id = "Complex_Balanced_Negative_Reward" 


param = { 
    
    "batch_size" : [30], #,50],#,50],# 30,50,60,[70]
    "target_update_freq" : [5000],#,40000],# 100000],  #10000], # 10000,20000,30000,[40000]
    "gamma" : [0.99],#,0.95],# # 0.98,0.99,0.995,[0.999]
    "train_freq" : [1],#,4], # 2,3,4,[5]
    "max_size" : [10000],#,70000], # 20000,50000,70000,[100000]
    "max_ep_length" : [1000],
    "policy" : ["linDecEpsGreedy","epsGreedy"],# "linDecEpsGreedy"],#, "epsGreedy"],
    "eps" : [0.3,0.1, 0.05],#, 0.3, 0.1],
    "delta_time" : [10,15],
    "reward" : ["balanced","negative"]
    #"optimizer": [optimizers.RMSprop(lr= 0.001), optimizers.Adagrad(), optimizers.Adam()]
}

param_grid = iter_params(**param)



def worker(input, output):
    """Runs through a chunk of the grid"""

    for position, args in iter(input.get, 'STOP'):
        result = worker_task(position, args)
        output.put(result)


def worker_task(position, args):
    """Tells the worker what to do with grid chunk"""
    # initialise all obje
    
    # print('Run', position + 1, '-- parameters', args)
    
    sumo_RL = simulation.simulator(
                    connection_label = position + 1,
                    q_network_type = 'simple',
                    target_q_network_type = 'simple',
                    gamma = args["gamma"],
                    target_update_freq = args["target_update_freq"],
                    train_freq = args["train_freq"],
                    num_burn_in = 1000,
                    batch_size = args["batch_size"],
                    optimizer = 'adam',
                    loss_func = "mse",
                    max_ep_length = args["max_ep_length"],
                    experiment_id = experiment_id,
                    model_checkpoint = True,
                    opt_metric = None,

                   # environment parameters
                    network = "complex", # complex
                    net_file = "complex_cross.net.xml", # "complex_cross.net.xml",
                    route_file = "cross.rou.xml",
                    demand = "rush",
                    state_shape = (1,41),#(1,41)
                    num_actions = 4, #4 
                    use_gui = False,
                    delta_time = args["delta_time"],
                    reward = args["reward"],

                   # memory parameters
                    max_size = args["max_size"],

                   # additional parameters

                    policy = args["policy"],
                    eps = args["eps"],
                    num_episodes = 1,
                    monitoring = True,
                    episode_recording = False,
                    hparams = args.keys())
    
    # print("training agent", position + 1)
    sumo_RL.train()
    # print("evaluating agent", position + 1)
    evaluation_results = sumo_RL.evaluate(runs = 5)

    return ({"run" : position + 1,
             "args" : args, 
             "eval_delay" : evaluation_results, 
             "eval_mean_delay" :evaluation_results["average_delay"]})


def gridsearch(param_grid):
    """Runs a parallelised gridsearch"""

    number_of_processes = multiprocessing.cpu_count()

    # Set up task list
    tasks = [(idx, val) for idx, val in enumerate(param_grid)]

    # Create queues
    task_queue = multiprocessing.Queue()
    done_queue = multiprocessing.Queue()

    # Submit tasks
    for task in tasks:
        task_queue.put(task)
    # Start worker processes
    for i in range(number_of_processes):
        print('Started process #', i + 1)
        multiprocessing.Process(target = worker,
                                args = (task_queue, done_queue)).start()
                    
    with open(os.path.join("./logs",experiment_id,"GS_results.json"), "a") as file:
            file.write('{ "results": [')
      
    # Get and print results
    results = []
    for i in range(len(tasks)):
        results.append(done_queue.get())
        
        with open(os.path.join("./logs",experiment_id,"GS_results.json"), "a") as file:
            json.dump(results[-1], file , indent=4) 
            if i != len(tasks)-1:
                file.write(",\n")
            
    with open(os.path.join("./logs",experiment_id,"GS_results.json"), "a") as file:
        file.write("]}")
                  
        #print('%s -- [RESULTS]: Run %s -- Parameters %s -- Mean duration %6.0f' % results[-1])
        
    # Tell child processes to stop
    for i in range(number_of_processes):
        task_queue.put('STOP')

    # Now combine the results
#     scores = [result[-1] for result in results]
#     lowest = min(scores)
#     winner = results[scores.index(lowest)]
#    return winner, results

multiprocessing.freeze_support()
gridsearch(param_grid)

Started process # 1
Started process # 2
Started process # 3
Started process # 4
Started process # 5
Started process # 6
Started process # 7
Started process # 8
Filling experience replay memory...
Filling experience replay memory...
Filling experience replay memory...
Filling experience replay memory...
Filling experience replay memory...
Filling experience replay memory...
Filling experience replay memory...
Filling experience replay memory...
...done filling replay memory
Run 7 -- running episode 1 / 1
...done filling replay memory
Run 3 -- running episode 1 / 1
...done filling replay memory
Run 1 -- running episode 1 / 1
...done filling replay memory
Run 8 -- running episode 1 / 1
...done filling replay memory
Run 6 -- running episode 1 / 1
...done filling replay memory
Run 2 -- running episode 1 / 1
...done filling replay memory
Run 4 -- running episode 1 / 1
...done filling replay memory
Run 5 -- running episode 1 / 1
Evaluate 3 -- running episode 1 / 5
Evaluate 4 -- running episod

/mnt/ca44e637-2dbd-4e16-beae-02318cbd0041/jrd/Documents/3. Institut, Universitat i Cursos/BGSE/4.Third_term/1.Master_Thesis/MS_Transports_RL/Scripts/Simple_Cross/cross_RL/environment.py:296: UserWarning: API change now handles step as floating point seconds
  fixed_con.simulationStep(self.max_ep_len*self.time_step)
/mnt/ca44e637-2dbd-4e16-beae-02318cbd0041/jrd/Documents/3. Institut, Universitat i Cursos/BGSE/4.Third_term/1.Master_Thesis/MS_Transports_RL/Scripts/Simple_Cross/cross_RL/environment.py:296: UserWarning: API change now handles step as floating point seconds
  fixed_con.simulationStep(self.max_ep_len*self.time_step)
/mnt/ca44e637-2dbd-4e16-beae-02318cbd0041/jrd/Documents/3. Institut, Universitat i Cursos/BGSE/4.Third_term/1.Master_Thesis/MS_Transports_RL/Scripts/Simple_Cross/cross_RL/environment.py:296: UserWarning: API change now handles step as floating point seconds
  fixed_con.simulationStep(self.max_ep_len*self.time_step)
/mnt/ca44e637-2dbd-4e16-beae-02318cbd0041/jrd/Doc

Evaluate 3 -- running episode 2 / 5


/mnt/ca44e637-2dbd-4e16-beae-02318cbd0041/jrd/Documents/3. Institut, Universitat i Cursos/BGSE/4.Third_term/1.Master_Thesis/MS_Transports_RL/Scripts/Simple_Cross/cross_RL/environment.py:296: UserWarning: API change now handles step as floating point seconds
  fixed_con.simulationStep(self.max_ep_len*self.time_step)
/mnt/ca44e637-2dbd-4e16-beae-02318cbd0041/jrd/Documents/3. Institut, Universitat i Cursos/BGSE/4.Third_term/1.Master_Thesis/MS_Transports_RL/Scripts/Simple_Cross/cross_RL/environment.py:296: UserWarning: API change now handles step as floating point seconds
  fixed_con.simulationStep(self.max_ep_len*self.time_step)


Evaluate 2 -- running episode 2 / 5
Evaluate 5 -- running episode 2 / 5
Evaluate 6 -- running episode 2 / 5
Evaluate 1 -- running episode 2 / 5
Evaluate 4 -- running episode 2 / 5
Evaluate 3 -- running episode 3 / 5
Evaluate 2 -- running episode 3 / 5
Evaluate 5 -- running episode 3 / 5


...

Evaluate 4 -- running episode 3 / 5
Evaluate 8 -- running episode 2 / 5
Evaluate 2 -- running episode 4 / 5
Evaluate 3 -- running episode 4 / 5
Evaluate 5 -- running episode 4 / 5
Evaluate 7 -- running episode 2 / 5
Evaluate 1 -- running episode 3 / 5
Evaluate 6 -- running episode 3 / 5
Evaluate 2 -- running episode 5 / 5
Evaluate 3 -- running episode 5 / 5
Evaluate 5 -- running episode 5 / 5
Evaluate 1 -- running episode 4 / 5
Evaluate 8 -- running episode 3 / 5
Evaluate 4 -- running episode 4 / 5
Filling experience replay memory...
Filling experience replay memory...
Filling experience replay memory...
Evaluate 6 -- running episode 4 / 5
...done filling replay memory
Run 9 -- running episode 1 / 1
Evaluate 1 -- running episode 5 / 5
...done filling replay memory
Run 10 -- running episode 1 / 1
Evaluate 4 -- running episode 5 / 5
Evaluate 7 -- running episode 3 / 5
...done filling replay memory
Run 11 -- running episode 1 / 1
Evaluate 9 -- running episode 1 / 5
Evaluate 10 -- running 

Process Process-2:
Process Process-1:
Process Process-3:
Process Process-8:
Process Process-6:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process Process-4:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/jrd/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/jrd/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/jrd/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/jrd/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-d868fd325202>", line 30, in worker
    result = worker_task(position, args)
Process Process-7:
  File "<ipython-input-2-d868fd325202>", line 82, in worker_task
    evaluation_results = sumo_RL.evaluate(runs = 5)
  File "/mn

  File "/mnt/ca44e637-2dbd-4e16-beae-02318cbd0041/jrd/Documents/3. Institut, Universitat i Cursos/BGSE/4.Third_term/1.Master_Thesis/MS_Transports_RL/Scripts/Simple_Cross/cross_RL/environment.py", line 355, in update_state
    self.compute_time_in_lane(connection, lane) # waiting time in lane
  File "/usr/share/sumo/tools/traci/connection.py", line 79, in _recvExact
    t = self._socket.recv(4 - len(result))
  File "<ipython-input-2-d868fd325202>", line 30, in worker
    result = worker_task(position, args)
  File "/mnt/ca44e637-2dbd-4e16-beae-02318cbd0041/jrd/Documents/3. Institut, Universitat i Cursos/BGSE/4.Third_term/1.Master_Thesis/MS_Transports_RL/Scripts/Simple_Cross/cross_RL/environment.py", line 251, in step
    self.take_action(action)
  File "/mnt/ca44e637-2dbd-4e16-beae-02318cbd0041/jrd/Documents/3. Institut, Universitat i Cursos/BGSE/4.Third_term/1.Master_Thesis/MS_Transports_RL/Scripts/Simple_Cross/cross_RL/simulation.py", line 210, in train
    self.ddqn.fill_replay(self.

  File "/usr/share/sumo/tools/traci/connection.py", line 134, in _sendReadOneStringCmd
    return self._checkResult(cmdID, varID, objID)
  File "/usr/share/sumo/tools/traci/connection.py", line 162, in _checkResult
    result = self._sendExact()
  File "/usr/share/sumo/tools/traci/connection.py", line 101, in _sendExact
    result = self._recvExact()
  File "/usr/share/sumo/tools/traci/connection.py", line 79, in _recvExact
    t = self._socket.recv(4 - len(result))
KeyboardInterrupt
  File "/usr/share/sumo/tools/traci/connection.py", line 79, in _recvExact
    t = self._socket.recv(4 - len(result))
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
curr_eps = 
total_it = 100
for itr in range(100)
    if itr < total_it:
        curr_eps = (curr_eps - init_eps) / total_it * itr + init_eps
    else :
        curr_eps = init_eps

In [ ]:
decay_test = []
eps_test = 0.8
for i in range(300000):
    eps_test *= omega_test ** i
    decay_test.append(eps_test)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(np.arange(300000),decay_test)

In [ ]:
first_results = pd.DataFrame(results).sort_values(by = 3)

In [ ]:
# Get max evaluate results

res = {}
with open('./logs/Test_evaluate/GS_results.json') as file:
    data = json.load(file)
    for run in data['results']:
        run_no = run["run"]
        res[f"{run_no}"] = run["eval_mean_delay"]
        
res

In [ ]:
pd.read_json('./logs/Test_evaluate/GS_results.json',)